In [1]:
import keras
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from PIL import Image
from tqdm import tqdm
import numpy as np
import scipy

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/paragag/persona/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/paragag/persona/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/paragag/persona/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/paragag/persona/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/paragag/persona/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtim

In [2]:
image_size = (331, 331, 3)
nas_model = keras.applications.nasnet.NASNetLarge(input_shape=image_size, include_top=False, weights='imagenet')

NameError: name 'keras' is not defined

In [ ]:
X_img = np.array(X_img)

In [ ]:
last_n = 18

for layer in nas_model.layers[:-last_n]:
    layer.trainable = False

In [ ]:
def get_model(feature_model, denses):
    inputs = feature_model.input
    features = feature_model.output
    x_1 = layers.GlobalAveragePooling2D()(features)
    x_2 = layers.GlobalMaxPooling2D()(features)
    x = layers.concatenate([x_1, x_2])

    y = layers.Dense(denses[0])(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    
    for i in range(1, len(denses) - 1):
        y = layers.Dense(denses[i])(y)
        y = layers.BatchNormalization()(y)
        y = layers.Activation('relu')(y)
    
    y = layers.Dense(denses[-1])(y)
    y = layers.Activation('softmax')(y)
    
    m = models.Model(inputs=inputs, outputs=y)
    opt = optimizers.Adam()
    m.compile(optimizer=opt, loss="categorical_crossentropy", metrics="accuracy")
    return m

In [ ]:
color_model = get_model(nas_model, denses=[64, 32, 5])
style_model = get_model(nas_model, denses=[256, 64, 32, 5])

In [ ]:
# Train color model

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        './data/color/train',
        target_size=image_size[:2],
        batch_size=64,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        './data/color/val',
        target_size=image_size[:2],
        batch_size=64,
        class_mode='binary')

color_model.fit_generator(
        train_generator,
        epochs=30,
        validation_data=validation_generator,)